# Contacts(0)

In dit notebook introduceren we een eenvoudige database voor contactgegevens van personen.
Deze contactgegevens verschillen sterk van persoon tot persoon: daardoor zijn deze eenvoudiger in een MongoDB-document te beschrijven dan in een relationele database.

Als voorbeeld-inhoud van de database gebruiken we het bestand adressen.json. 
Dit voorbeeld geeft ook een goed inzicht in de grote verschillen tussen de contact-documenten.

## Hoe gebruik je dit Notebook?

* een notebook bestaat uit cellen; een cel kan *tekst* bevatten (Markdown), zoals deze cel, of (Python) *code*, zoals de cellen hieronder.
* je voert een cel uit door deze te selecteren (cursor in de cel), en vervolgens SHIFT-RETURN in te toetsen.
  Ook het pijltje in de opdrachtenbalk hierboven kun je gebruiken.
* onder de cel zie je dan de uitvoer van deze opdracht.
* alle variabelen enz. die je introduceert in de code van een cel kun je in de volgende cellen gebruiken.
* met een `!` kun je een shell-opdracht uitvoeren; het resultaat kun je in Python gebruiken (als string).
* om problemen te voorkomen voer je cellen alleen uit *in de volgorde in het notebook*.
* je kunt eventueel opnieuw beginnen door de "Kernel" opnieuw te starten (via het cirkeltje, als bij een reload in de browser).
* zie voor meer informatie: help, en [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

De meeste code-cellen kun je zo uitvoeren; probeer de code en de uitvoer te begrijpen.
Bij sommige opdrachten moet je de code aanpassen, en dan de cel uitvoeren.

Voer nu de cellen hieronder uit.

In [ ]:
print("Hello World")

In [ ]:
3 + 4

In [ ]:
filenames = !ls
print(filenames)

## Pymongo

pymongo is de "driver" voor MongoDB vanuit Python.
De documentatie hiervan vind je via: https://api.mongodb.com/python/current/
We gaan hier uit van de situatie waarbij zowel MongoDB als pymongo al geïnstalleerd zijn.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

## Databasenaam

We leiden de naam van de database af van de naam van de gebruiker.
Deze naam krijgen we via de shell-opdracht `echo $USER`.
Door deze naam te gebruiken voorkomen we dat de gebruikers van hetzelfde database-managament-systeem elkaar in de weg zitten.

In [ ]:
userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
dbname

## Verbinding met de database

De eerste stap is om verbinding te maken met MongoDB, en met de *demo*-database.
In dit geval hebben we geen speciale autorisatie nodig; in de praktijk is dat meestal wel nodig.
In deze database gebruikenn we voorlopig maar één collection: *contacts*

In [ ]:
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

## Inlezen van de voorbeeld-data

We importeren de voorbeeld-data uit een tekstbestand in JSON-formaat.
Dit tekstformaat is geschikt voor het uitwisselen van *objecten*.
Voor allerlei programmeertalen bestaan er libraries om objecten in dit formaat in te lezen of weg te schrijven.

**Opdracht** bekijk het bestand addressen.json, met behulp van de opdracht `!cat adressen.json` in de onderstaande code-cel.
Geef een voorbeeld van de contactgegevens van een persoon in dit formaat, met telefoonnummer, adres, en email-adres.

**Opdracht** hoe zou je aan kunnen geven dat een persoon meerdere telefoonnummers heeft? 
(NB: dit is lastig; hier komen we later op terug.)

Via de (shell)opdracht `mongoimport` importeren we de voorbeeld-data in de collection `contacts` van de database `demo`.
Als de opdracht gelukt is krijg je 0 als resultaat; een andere waarde geeft een foutcode aan.

In [ ]:
mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

collection.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Query: alle elementen

De eenvoudigste zoekopdracht is om alle documenten in de collection te vinden.
Deze drukken we vervolgens af.

> We kunnen het resultaat ook als Python-lijst krijgen, maar de *cursor*-constructie is geschikter voor grote hoeveelheden documenten.

> Het `_id`-veld is de *key* van het document (record) in de collection.

In [ ]:
cursor = collection.find()
for obj in cursor:
    print(obj)